In [1]:
import numpy

# generate cartesian product of multiple array
# input: tuple of arrays
def cartesian_product(arrays):
    broadcastable = numpy.ix_(*arrays)
    broadcasted = numpy.broadcast_arrays(*broadcastable)
    rows, cols = reduce(numpy.multiply, broadcasted[0].shape), len(broadcasted)
    out = numpy.empty(rows * cols, dtype=broadcasted[0].dtype)
    start, end = 0, rows
    for a in broadcasted:
        out[start:end] = a.reshape(-1)
        start, end = end, end + rows
    return out.reshape(cols, rows).T

In [2]:
import numpy
import numpy.random as rand
import math

from WaveBlocksND import *
from sample_wavepacket import *
from yaml_writer import *

print("")
print("--- Create sample wavepacket ---")

D = 3
shape = LimitedHyperbolicCutShape(D,7.0,(5,5,5))
packet = createSampleWavepacket(D, shape)
yaml_write_hawp(packet, "wavepacket")

print("")
print("--- Evaluate wavepacket at test location ---")
x0 = numpy.zeros((D,1),dtype=complex)
for d in range(0,D):
    x0[d,0] = (d+1)/float(2*D) + (D-d)/float(2*D)*1.0j
print("psi: ", packet.slim_recursion(x0,0))
print("psi (with prefactor): ", packet.slim_recursion(x0,0,True))

for index, phi, coeff in zip(list(packet.get_basis_shapes(0).get_node_iterator()), packet.evaluate_basis_at(x0,0), packet.get_coefficients(0)):
    print(index, phi, coeff)

print("")
print("--- Evaluate wavepacket at some random locations ---")
rand.seed(0)
points = rand.randn(D,20)

results = packet.slim_recursion(points,0)[0]

import csv
with open("wavepacket_evaluation.csv", "w") as filehandle:
    csvfile = csv.writer(filehandle, delimiter=' ')
    for point,result in zip(points.T,results):
        csvfile.writerow(tuple(point) + (numpy.real(result),) + (numpy.imag(result),))

print("")
print("--- Done ---")



--- Create sample wavepacket ---

--- Evaluate wavepacket at test location ---
('psi: ', array([[-0.12870212+0.11237119j]]))
('psi (with prefactor): ', array([[-0.13089496+0.10870531j]]))
((0, 0, 0), array([ 0.18100493+0.1360191j]), array([ 0.03097689+0.02580412j]))
((0, 0, 1), array([ 0.49808765+0.46265537j]), array([ 0.02909289+0.00967338j]))
((0, 0, 2), array([ 0.83953782+1.03779092j]), array([ 0.0154223+0.00683246j]))
((0, 0, 3), array([ 0.89834962+1.73174241j]), array([ 0.00242399+0.01550085j]))
((0, 0, 4), array([ 0.38607828+2.18867146j]), array([-0.00015523+0.02470947j]))
((0, 1, 0), array([ 0.11986951+0.39978984j]), array([ 0.03394277+0.00834695j]))
((0, 1, 1), array([ 0.34777223+1.14192608j]), array([ 0.03167528-0.00482116j]))
((0, 1, 2), array([ 0.63415327+2.13140739j]), array([ 0.01879642-0.00609999j]))
((0, 2, 0), array([-0.21991398+0.36940539j]), array([ 0.02307543-0.00059142j]))
((0, 2, 1), array([-0.39341436+1.02416771j]), array([ 0.02175051-0.01173141j]))
((0, 3, 0), a

yaml_writer.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
  "q": q.astype(float).T[0].tolist(),
yaml_writer.py:19: ComplexWarning: Casting complex values to real discards the imaginary part
  "p": p.astype(float).T[0].tolist(),
